In [63]:
import networkx as nx
import csv
from tqdm import tqdm

ratings_file = '/home/boyto/DataSets/MovieLensResearch/ratings.csv'
MAX_RATINGS = 100000

G=nx.Graph()
i=0
with open(ratings_file, 'rb') as csvfile:
    reader = csv.reader(csvfile)
    line1 = reader.next()
    for j in tqdm(range(0,MAX_RATINGS)):
        row = reader.next()
        #print row            
        G.add_edge('u'+row[0],'m'+row[1])
        if i>MAX_RATINGS:
            break
        i = i +1
#print(list(G.nodes()))
#print(list(G.edges()))

100%|██████████| 100000/100000 [00:00<00:00, 185925.15it/s]


In [133]:
#starting_node = 'm296' #m296= Pulp Fiction
starting_node = 'm1' 
NUM_WALKS = 100000
BETA = 0.8

node = starting_node
users_table = dict()
movies_table = dict()
import random
import operator

for i in tqdm(range(0,NUM_WALKS)):
    r_beta = random.random()
    if r_beta < BETA:
        # Choose a random neighbor
        node = random.choice(G.neighbors(node))
    else:
        # Go back again!
        node = starting_node
    if node != starting_node:
        if node[0]=='u':
            if node in users_table:
                users_table[node]+=1
            else:
                users_table[node]=1
        else:
            if node in movies_table:
                movies_table[node]+=1
            else:
                movies_table[node]=1
    #print node
movies_table = sorted(movies_table.items(), key=operator.itemgetter(1),reverse=True)
users_table = sorted(users_table.items(), key=operator.itemgetter(1),reverse=True)
    
print movies_table[0:10]

100%|██████████| 100000/100000 [00:01<00:00, 80452.26it/s]

[('m356', 163), ('m780', 156), ('m296', 149), ('m260', 131), ('m110', 128), ('m318', 127), ('m150', 123), ('m32', 118), ('m480', 118), ('m592', 115)]


# Think the LSH thingie for movies and users!!!

In [109]:
_memomask = {}

b = 20
r = 3
NUM_MINHASHES = b*r
SHINGLE_LENGTH = 8
HASH_SPACE = 2**20

In [122]:
LSH_tables = [dict() for i in range(b)]

In [123]:
def get_hash_function(n):
  mask = _memomask.get(n)
  if mask is None:
    random.seed(n)
    mask = _memomask[n] = random.getrandbits(32)
  def myhash(x):
    return hash(x) ^ mask
  return myhash

In [124]:
def shingles(string, length):
    return (string[0+i:length+i] for i in range(0, len(string)-length+1, 1))

In [125]:
def get_minhashes(string):
    minhashes = [float("inf")]*NUM_MINHASHES
    for sh in shingles(string, SHINGLE_LENGTH):
        for i in range(NUM_MINHASHES):
            mh = hash_functions[i](sh) % HASH_SPACE
            if mh < minhashes[i]:
                minhashes[i] = mh
    return minhashes

In [126]:
def store_tweet(line,index):
    # Compute minhashes
    mh = get_minhashes(line.lower())
    # Now group minhashes in b groups of r minhashes each
    for i in range(b):
        hash_key = ''.join(map(str,(mh[i*r:(i*r)+r])))
        if hash_key not in tables[i]:
            tables[i][hash_key]=[index]
        else:
            print all_tweets[index]
            print "collides with:"
            for tweet in tables[i][hash_key]:
                print all_tweets[tweet]
            tables[i][hash_key].append(index)
            print "---------------------------"


In [127]:
hash_functions = [get_hash_function(i) for i in range(0,NUM_MINHASHES)]

In [128]:
# A dictionary of Minhashes for every movie
movie_mh = dict()
discretize_ratings = {'0.0':'b','0.5':'b','1.0':'b','1.5':'b','2.0':'b',\
                      '2.5':'n','3.0':'n','3.5':'n',\
                      '4.0':'g','4.5':'g','5.0':'g'}

In [129]:
ratings_file = '/home/boyto/DataSets/MovieLensResearch/ratings.csv'
MAX_RATINGS = 1000000

i=0
with open(ratings_file, 'rb') as csvfile:
    reader = csv.reader(csvfile)
    line1 = reader.next()
    for j in tqdm(range(0,MAX_RATINGS)):
        row = reader.next()
        movieId=row[1]
        if not movieId in movie_mh:
            movie_mh[movieId] = [float("inf")]*NUM_MINHASHES
        string_to_hash = row[0]+discretize_ratings[row[2]]
        for i in range(NUM_MINHASHES):
            mh = hash_functions[i](string_to_hash) % HASH_SPACE
            if mh < movie_mh[movieId][i]:
                movie_mh[movieId][i] = mh
        if i>MAX_RATINGS:
            break
        i = i +1


100%|██████████| 1000000/1000000 [01:18<00:00, 12765.24it/s]


In [130]:
collisions = 0
# Now that we have minhashes for every movie we have to store the movieIds in the LSH table
for movie_id in tqdm(movie_mh):
    mh = movie_mh[movie_id]
    for i in range(b):
        hash_key = ''.join(map(str,(mh[i*r:(i*r)+r])))
        if hash_key not in LSH_tables[i]:
            LSH_tables[i][hash_key]=[movie_id]
        else:
            LSH_tables[i][hash_key].append(movie_id)
            collisions = collisions+1
print "Number of Collisitions:"+str(collisions)

100%|██████████| 13950/13950 [00:01<00:00, 8610.30it/s]

Number of Collisitions:54257


In [132]:
target_movie = '296'
#target_movie = '1'
mh = movie_mh[target_movie]
similar_movies=dict()
for i in range(b):
        hash_key = ''.join(map(str,(mh[i*r:(i*r)+r])))
        if hash_key in LSH_tables[i]:
            for candidate in LSH_tables[i][hash_key]:
                if candidate != target_movie:
                    if candidate in similar_movies:
                        similar_movies[candidate]+=1
                    else:
                        similar_movies[candidate]=1
            #similar_movies.append(tables[i][hash_key])
similar_movies = sorted(similar_movies.items(), key=operator.itemgetter(1),reverse=True)
print similar_movies            

[('50', 4), ('150', 2), ('593', 2), ('4963', 1), ('3578', 1), ('5445', 1), ('2858', 1), ('1263', 1), ('2762', 1), ('1193', 1), ('1206', 1), ('4886', 1), ('1197', 1), ('356', 1), ('1089', 1), ('47', 1), ('2028', 1), ('293', 1), ('1673', 1)]
